In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw4.ipynb")

# HW4: Experimental Design - Power, SUTVA, and Sampling Bias

In this homework, you will analyze two real pricing experiments to understand critical aspects of experimental design:
- How sample size affects statistical power
- What happens when SUTVA (Stable Unit Treatment Value Assumption) is violated
- How sampling bias can distort treatment effect estimates

You'll work with data from two different retailers testing price promotions.

In [ ]:
import polars as pl
import numpy as np
import statsmodels.api as sm
from plotnine import *

## Part 1: Statistical Power and Sample Size

### Background Story

You work as a data analyst for **ElectroHub**, a national electronics retailer with 1,000 locations. The merchandising team wants to test whether a $5 price discount on two popular products (wireless headphones and bluetooth speakers) increases sales volume. 

At the baseline price of $200, the average daily sales per store is 20 units for both products.

Stores were randomly assigned to either:
- **Control**: Keep the regular $200 price
- **Treatment**: Offer a $195 promotional price

The team tracked daily unit sales for both products during a one-week test period. They want to understand how sample size affects the reliability of the results.

**Question 1 (5 pts):** Before analyzing the data, the team wants to calculate the required sample size for adequate statistical power.

Use an online power calculator ([https://www.statskingdom.com/sample_size_regression.html](https://www.statskingdom.com/sample_size_regression.html)) and compute the necessary sample size with:

- Target power = 80%
- Significance level (α) = 0.05
- Effect size = 0.1 
- Predictors = 1 (treatment indicator only)

Store your answer as an integer in `sample_size_power`.

In [ ]:
sample_size_power = ...

print(f"Required sample size for 80% power: {sample_size_power}")

In [ ]:
grader.check("q1")

Now let's load the actual experimental data and see how sample size affects the results.

In [ ]:
# Load the experimental data
df = pl.read_csv("hw4_data1.csv")
df.head(10)

The dataset contains:
- `store_id`: Unique store identifier
- `treatment`: 1 if store got the $195 promotional price, 0 if regular $200 price
- `price`: Actual price charged ($195 or $200)
- `sales_A`: Daily unit sales for Product A (wireless headphones)
- `sales_B`: Daily unit sales for Product B (bluetooth speakers)

**Question 2 (10 pts):** To understand how sample size affects statistical power, run regressions at different sample sizes.

Create three subsets of the data:
- **Small**: First 50 stores
- **Medium**: First 250 stores  
- **Large**: First 1000 stores (all data)

For each sample size, run **two regressions**:
1. `sales_A ~ treatment` (Product A)
2. `sales_B ~ treatment` (Product B)

Store the **6 fitted model objects** in variables named:
- `model_A_small`, `model_A_medium`, `model_A_large`
- `model_B_small`, `model_B_medium`, `model_B_large`

We'll use these to extract coefficients and p-values.

In [ ]:
# Create subsets
df_small = ...
df_medium = ...
df_large = ...

# Helper function to run regression
def run_regression(data, outcome_var):
    """Run OLS regression: outcome ~ treatment"""
    X = sm.add_constant(data['treatment'].to_numpy())
    y = data[outcome_var].to_numpy()
    model = sm.OLS(y, X).fit()
    return model

# Product A regressions
model_A_small = ...
model_A_medium = ...
model_A_large = ...

# Product B regressions
model_B_small = ...
model_B_medium = ...
model_B_large = ...

# Extract results into a table for comparison
results = []
for name, model in [
    ('A_small', model_A_small),
    ('A_medium', model_A_medium),
    ('A_large', model_A_large),
    ('B_small', model_B_small),
    ('B_medium', model_B_medium),
    ('B_large', model_B_large)
]:
    results.append({
        'model': name,
        'coef': model.params[1],
        'stderr': model.bse[1],
        'pvalue': model.pvalues[1]
    })

results_df = pl.DataFrame(results)
print("\nRegression Results by Sample Size:")
print(results_df)

In [ ]:
grader.check("q2")

**Question 3 (10 pts):** Based on the regression results, let's evaluate the profitability of the promotion for Product A.

Given:
- The price was reduced by $5 (from $200 to $195)
- Product A costs $80 to manufacture and ship

Looking at your regression results for Product A:
- The treatment coefficient represents the **additional units sold** per store per day due to the $5 discount
- The baseline sales per store per day is 20 units

**True or False**: Based on the large sample regression results (1000 stores), implementing the $5 discount for Product A would be profitable.

Store your answer in `treatment_profitable_A` as `True` or `False`.

**Hint**: Calculate the profit in the baseline (assume 20 units per day) and the profit under the treatment price.

In [ ]:
# Extract key values from large sample model for Product A
treatment_effect_A = model_A_large.params[1]  # Additional units sold
baseline_sales = 20  # Baseline sales per store per day

# Calculate profits under each price
baseline_profit = ...
treatment_profit_A = ...
treatment_profitable_A = ...
    
print("\nProfitability Analysis for Product A:")
print(f"Baseline profit: ${baseline_profit:.2f}")
print(f"Treatment profit: ${treatment_profit_A:.2f}")
print(f"Product A profitability: {treatment_profitable_A}")

In [ ]:
grader.check("q3")

**Question 4 (5 pts):** Now consider Product B. Based on your power calculation in Question 1, at what sample size should we feel confident that there is no increase in sales of at least 10% from the $5 discount for Product B?

- A) Small sample (50 stores)
- B) Medium sample (250 stores)
- C) Large sample (1000 stores)

Store your answer as a string `'A'`, `'B'`, or `'C'` in `q4_answer`.

In [ ]:
q4_answer = ...

print(f"Answer: {q4_answer}")

In [ ]:
grader.check("q4")

## Part 2: SUTVA Violation and Spillover Effects

### Background Story

You're a pricing analyst at **Volt Energy**, a regional energy drink brand sold through 600 convenience stores across 150 districts (4 stores per district). Districts are relatively small geographic areas where any given customer is close enough to visit multiple different stores in the same district. But different districts are far apart, so customers would rarely travel between districts.

The marketing team ran a pricing experiment:
- **Control**: Regular price of $3.00 per can
- **Treatment**: Promotional price of $2.95 per can

You expect the baseline sales to be 500 cans per store per week at the regular price.

The experiment was designed with district-level randomization, but there's a problem: in some districts, the implementation was inconsistent. Some districts had all stores following the same pricing (either all control or all treatment), while other "mixed" districts had a combination of control and treatment stores.

In [ ]:
# Load the experiment data
df2 = pl.read_csv("hw4_data2.csv")
baseline_sales = 500

The dataset contains:
- `store_id`: Unique store identifier
- `district_id`: District identifier (4 stores per district)
- `treatment`: 1 if store offered $2.95 promotional price, 0 if regular $3.00
- `price`: Actual price charged
- `avg_district_income`: Average household income in the district
- `sales`: Weekly energy drink sales (cans)

**Question 5 (10 pts):** Start with a naive analysis that ignores potential spillover issues.

Run a simple regression using **all stores**: `sales ~ treatment`

Use the `statsmodels` library to fit the model: `sm.OLS(y,X).fit()`

Store the fitted model object in `naive_model` and extract the treatment coefficient as `naive_coef`.

**Hint**: You can extract the value of a coefficient using `model.params['variable_name']`.

In [ ]:
# Run naive regression on all stores
X_naive = sm.add_constant(df2['treatment'].to_pandas())
y_naive = df2['sales'].to_pandas()
naive_model = ...

naive_model.summary()

In [ ]:
naive_coef = ...
print(f"Naive treatment effect: {naive_coef:.2f} cans per week")
print(f"Percentage effect: {(naive_coef / baseline_sales):.2%}")

In [ ]:
grader.check("q5")

Given the potential for spillover effects, we plot the distribution of treatment rates across districts to see the proportion of treated units within each district.

In [ ]:
(ggplot(df2.group_by('district_id').agg(pl.col('treatment').mean().alias('treatment_rate')), aes(x="treatment_rate"))
+ geom_histogram(aes(fill=after_stat('x')), bins=3, show_legend=False)
+ scale_fill_gradientn(colors=["#377eb8", "#999999", "#ff7f00"]) # Blue, Gray, Orange
+ labs(title="Distribution of Treatment Rates", x="Treatment Rate", y="Frequency"))

It appears a significant number of districts have mixed treatment assignments!

**Question 6 (10 pts):** 

The dataframe helpfully contains a `district_type` column with the following values:
- `"pure_control"`: 0 treated stores (all 4 stores are control)
- `"mixed"`: 1, 2, or 3 treated stores (mixture of treatment and control)
- `"pure_treatment"`: 4 treated stores (all 4 stores are treatment)

Compare treatment effects in pure vs mixed districts to detect spillover by running separate regressions for each group.

Store your results:
- `pure_coef`: Treatment coefficient in pure districts (in cans per week)
- `mixed_coef`: Treatment coefficient in mixed districts (in cans per week)

If spillover is present, we'd expect the mixed district coefficient to be larger (or smaller) than the pure district coefficient due to contamination.

**Hint**: For each group, calculate the mean sales for treatment stores and control stores separately, then take the difference.

In [ ]:
# Pure districts analysis
df_pure = df2.filter(
    ...
)

X_pure = sm.add_constant(df_pure['treatment'].to_pandas())
y_pure = df_pure['sales'].to_pandas()
pure_model = ...
pure_model.summary()


In [ ]:
# Mixed districts analysis
df_mixed = df2.filter(  
    ...
)  

X_mixed = sm.add_constant(df_mixed['treatment'].to_pandas())
y_mixed = df_mixed['sales'].to_pandas()
mixed_model = ...
mixed_model.summary()

In [ ]:
pure_coef = ...
mixed_coef = ...
print(f"Pure districts treatment effect: {pure_coef:.2f} cans per week")
print(f"Percentage effect in pure districts: {(pure_coef / baseline_sales):.2%}")
print(f"Mixed districts treatment effect: {mixed_coef:.2f} cans per week")
print(f"Percentage effect in mixed districts: {(mixed_coef / baseline_sales):.2%}")

In [ ]:
grader.check("q6")

<!-- BEGIN QUESTION -->

**Question 7 (10 pts, open ended):** Based on your results from Question 6, do you observe evidence of spillover effects in the mixed districts? If so, explain why spillover might be occurring and how it might bias the treatment effect estimate.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Part 3: Sampling Bias and Regression Control

Even after filtering to pure districts to remove spillover, there's another problem: the treatment assignment wasn't truly random. The marketing team's randomization process was flawed—income level influenced which districts were assigned to treatment vs control.

This creates **sampling bias** that can distort our treatment effect estimates.

**Question 8 (10 pts):** Investigate whether income is correlated with treatment assignment.

Using the **pure districts data** (from Question 6), calculate the correlation between `avg_district_income` and `treatment` assignment.

Store the correlation coefficient in `income_treatment_corr`.

A non-zero correlation would indicate sampling bias—certain income groups were more likely to be assigned to treatment.

**Hint**: Given a polars dataframe `df`, you can get the correlation matrix for a list of columns `['col1', 'col2']` using `df[['col1', 'col2']].corr()`

In [ ]:
df_pure[['avg_district_income', 'treatment']].corr()[0,1]

In [ ]:
# Calculate correlation between income and treatment in pure districts
income_treatment_corr = ...

print(f"Correlation between income and treatment assignment: {income_treatment_corr:.4f}")
if abs(income_treatment_corr) > 0.1:
    print(f"This indicates sampling bias - income is confounded with treatment!")
else:
    print(f"Treatment assignment appears random with respect to income.")

In [ ]:
grader.check("q8")

**Question 9 (10 pts):** Remove sampling bias by controlling for income in a regression.

Using the **pure districts data**, run a regression with income control:
`sales ~ treatment + avg_district_income`

Store the fitted model in `controlled_model` and extract:
- `controlled_coef`: The treatment coefficient

This controlled estimate should recover the true treatment effect by removing the confounding influence of income.

In [ ]:
# Regression with income control (pure districts only)
X_controlled = ...
y_controlled = ...
controlled_model = ...

controlled_model.summary()

In [ ]:
controlled_coef = ...

In [ ]:
grader.check("q9")

**Question 10 (10 pts):** Evaluate the business impact of getting the treatment effect estimate right.

Given:
- Cost per can: $2.20
- Control price: $3.00 per can
- Treatment price: $2.95 per can
- Baseline sales: we know average sales per store per week is **500 cans**

Calculate the **profits** under:
- The baseline price and demand
- The treatment price and the **naive estimate of demand** (from Question 5): Using all stores without any corrections
- The treatment price and the **corrected estimate** (from Question 9): Using pure districts with income control

Then decide whether the implementation of the $2.95 price is profitable under each estimate.

Store:
- `profit_baseline`: The baseline profit per store per week
- `profit_treatment_naive`: Weekly profit change per store using naive estimate
- `profit_treatment_corrected`: Weekly profit change per store using corrected estimate
- `profitable_change_naive`: Whether the naive estimate indicates profitability (True/False)
- `profitable_change_corrected`: Whether the corrected estimate indicates profitability (True/False)

This shows how biased estimates affect business decisions.

In [ ]:
# Given parameters
cost_per_can = 2.20
control_price = 3.00
treatment_price = 2.95
baseline_sales = 500

# Baseline profit
profit_baseline = ...

# Profit under naive estimate
naive_treatment_effect = naive_coef # From Q5
profit_treatment_naive = ...
profitable_change_naive = ...

# Profit under corrected estimate
corrected_treatment_effect = controlled_coef # From Q9
profit_treatment_corrected = ...
profitable_change_corrected = ...

print("\nProfitability Analysis:")
print(f"Baseline profit: ${profit_baseline:.2f}")
print(f"Treatment profit (naive estimate): ${profit_treatment_naive:.2f}, Profitable: {profitable_change_naive}")
print(f"Treatment profit (corrected estimate): ${profit_treatment_corrected:.2f}, Profitable: {profitable_change_corrected}")

In [ ]:
grader.check("q10")

**Question 11 (10 pts):** In Part 3, we found that the treatment assignment was systematically correlated with income (sampling bias), which distorted our naive estimate.

Which of the following approaches would **NOT** be an effective strategy to resolve such **sampling bias**?

- A) **Stratified Randomization**: Grouping stores by income level first, then randomizing within those groups to ensure balance.
- B) **Increasing Sample Size**: Adding more stores to the experiment using the original assignment procedure.
- C) **Multivariate Regression**: Including the confounding variable (e.g., income) as a control in the analysis to isolate the treatment effect.

Store your answer as a string `'A'`, `'B'`, or `'C'` in `q11_answer`.

In [ ]:
q11_answer = ...

print(f"Answer: {q11_answer}")

In [ ]:
grader.check("q11")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have the zip file, upload the **entire** zip file to Gradescope for grading.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)